<a href="https://colab.research.google.com/github/jabarragann/PytorchTutorialsJuanAntonio/blob/master/EEG_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Google drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# !ls
# !mkdir -p 'gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/D4/'
# !mkdir -p 'gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/D4/preprocess'
!ls
import os
os.chdir('/content')
!ls 

#!pip install pycuda

gdrive	sample_data
gdrive	sample_data


#Setting up the Environment


1. Delete current run time directory
2. Create all needed directories
3. Copy data from google drive account to Colab directory





In [5]:
import os
import shutil

def createDirectories(rootPath, *args):
      
    print("Creating directories")
    for idx, path in enumerate(args):
        try:  
            os.makedirs(rootPath + path)
        except OSError:  
            print ("{:d}: Creation of the directory {} failed".format(idx,path))
        else:  
            print ("{:d}: Successfully created the directory {}".format(idx, path))
    print()

def copyFiles(srcRootPath,dstRootPath, *directories):
    print("copy data from google drive")
    for idx, path in enumerate(directories):
        print("{:d}: Copy files from: {:s}".format(idx,path) )
        completePath = srcRootPath+path
        srcFiles = os.listdir(completePath)
        srcFiles.sort()

        for file in srcFiles:
            srcPath = completePath + file
            dstPath = dstRootPath+path

            if os.path.isfile(srcPath):
                shutil.copy(srcPath, dstPath)

    print()

#Set /content as working directory
os.chdir('/content')

#Remove Previous RuntimeDirectory
try:
    shutil.rmtree('./RunTimeDir')
except:
    print("RunTimeDirectory not found")
else:
    print("Deleting previous RunTimeDirectory")
finally:
    print()


# Create new directories
rootPath = "./RunTimeDir/Data/"
directories = ['D1/preprocessed','D1/final', 'D1/raw',
               'D2/preprocessed','D2/final', 'D2/raw',
               'D3/preprocessed','D3/final', 'D3/raw',
               'D4/preprocessed','D4/final', 'D4/raw',]

createDirectories(rootPath, *directories)


#Copy data from google drive to RuntimeDir
gdrivePath ='gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Data/'
srcDirectories = ['D1/raw/', 'D2/raw/', 'D3/raw/','D4/raw/']

dstRootPath = "./RunTimeDir/Data/"

copyFiles(gdrivePath, dstRootPath, *srcDirectories)


# Check current working directory
os.chdir('/content/RunTimeDir')
path = os.getcwd()  
print ("The current working directory is %s" % path, end='\n\n') 



RunTimeDirectory not found

Creating directories
0: Successfully created the directory D1/preprocessed
1: Successfully created the directory D1/final
2: Successfully created the directory D1/raw
3: Successfully created the directory D2/preprocessed
4: Successfully created the directory D2/final
5: Successfully created the directory D2/raw
6: Successfully created the directory D3/preprocessed
7: Successfully created the directory D3/final
8: Successfully created the directory D3/raw
9: Successfully created the directory D4/preprocessed
10: Successfully created the directory D4/final
11: Successfully created the directory D4/raw

copy data from google drive
0: Copy files from: D1/raw/
1: Copy files from: D2/raw/
2: Copy files from: D3/raw/
3: Copy files from: D4/raw/

The current working directory is /content/RunTimeDir



#Data Preprocessing

Fuse Timestamps with sensor data


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import shutil


def preprocessData(trial=None, dataset=None):
    print("Preprocess Data from trial {:d}".format(trial))

    #Read Raw files
    epoc = pd.read_csv("./Data/D{:d}/raw/S1_T{:d}_Epoc.txt".format(dataset,trial), sep=' ')
    shimmer = pd.read_csv("./Data/D{:d}/raw/S1_T{:d}_Shimmer.txt".format(dataset,trial), sep=' ')
    secondaryTaskStamps = pd.read_csv("./Data/D{:d}/raw/S1_T{:d}_Timestamps.txt".format(dataset, trial), sep=' ', skiprows=[0,1,2,3])
    
    fiveSecondStampsSrc ="./Data/D{:d}/raw/S1_T{:d}_TimestampEvery5Seconds.txt".format(dataset,trial)
    fiveSecondStampsDst ="./Data/D{:d}/preprocessed/S1_T{:d}_TimestampEvery5Seconds.txt".format(dataset,trial)
    
    fiveSecondWindowStamps = pd.read_csv(fiveSecondStampsSrc, sep=' ')
    
    #Copy 5seconds timestamp to preprocessed file
    shutil.copy(fiveSecondStampsSrc, fiveSecondStampsDst)
    
    #Get Data Columns
    epocSlice = epoc[['AF3', 'F7', 'F3', 'FC5', 'T7',
                      'P7', 'O1', 'O2', 'P8', 'T8',
                      'FC6', 'F4', 'F8', 'AF4', 'COMPUTER_TIME']]
    epocArr = epocSlice.values

    shimmerSlice = shimmer[['PPG','COMPUTER_TIME']]
    shimmerArr = shimmerSlice.values

    secondaryTaskArr = secondaryTaskStamps.values
    windowStampsArr = fiveSecondWindowStamps.values

    finalArrayEpoc = np.ones((epocArr.shape[0], epocArr.shape[1] + 3)) * -10
    finalArrayEpoc[:, list(range(15))] = epocArr

    finalArrayShimmer = np.ones((shimmerArr.shape[0], shimmerArr.shape[1] + 3)) * (-10)
    finalArrayShimmer[:, list(range(2))] = shimmerArr

    #Apply a band pass Butterworth Filter to each of the 14 channels of the EEG
    from scipy.signal import butter, lfilter

    def butter_bandpass(lowcut, highcut, fs, order=7):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = butter(order, [low, high], btype='band')
        return b, a

    def butter_bandpass_filter(data, lowcut, highcut, fs, order=7):
        b, a = butter_bandpass(lowcut, highcut, fs, order=order)
        y = lfilter(b, a, data)
        return y

    #EPOC Filtering
    lowCut  = 0.5
    highCut = 50
    for i in range(14):
        temp = butter_bandpass_filter(finalArrayEpoc[:, i], lowCut, highCut, 128, order=7)
        finalArrayEpoc[:, i] = temp

    #Shimmer Filtering
    temp = butter_bandpass_filter(finalArrayShimmer[:, 0], lowCut, highCut, 128, order=7)
    finalArrayShimmer[:, 0] = temp

    #Remove 10 first seconds of data to take away outliers and transient state created by the band pass filter
    tempFinal = np.ones((epocArr.shape[0] - 700 , epocArr.shape[1]+3)) * -10
    tempFinal = finalArrayEpoc[700:, :]
    finalArrayEpoc = tempFinal

    tempFinal = np.ones((shimmerArr.shape[0] - 700 , shimmerArr.shape[1]+3)) * -10
    tempFinal = finalArrayShimmer[700:, :]
    finalArrayShimmer = tempFinal

    #### EEG PRE PROCESSING ####
    #Add Secondary task timestamps to EEG data
    idx = 0
    for i in range(secondaryTaskArr.shape[0]):
        timestamp = secondaryTaskArr[i,0]
        while timestamp > epocArr[idx,14]:
            idx += 1
            if idx == finalArrayEpoc.shape[0]:
                break

        if idx == finalArrayEpoc.shape[0]:
            break
        if idx < finalArrayEpoc.shape[0]:
            finalArrayEpoc[idx, -3] = timestamp

    #Add 5 Seconds Windows timestamps to EEG data
    idx = 0
    for i in range(windowStampsArr.shape[0]):
        timestamp = windowStampsArr[i,0]
        while timestamp > epocArr[idx,14]:
            idx += 1
            if idx == finalArrayEpoc.shape[0]:
                break

        if idx == finalArrayEpoc.shape[0]:
            break
        if idx < finalArrayEpoc.shape[0]:
            finalArrayEpoc[idx, -2] = timestamp

    # Add Label
    count = 0
    for idx in range(finalArrayEpoc.shape[0]):
        time = finalArrayEpoc[idx, 14]

        if count == secondaryTaskArr.shape[0] -1:
            finalArrayEpoc[idx, -1] = 1 if secondaryTaskArr[count, 1] is True else 0
        else:
            if time > secondaryTaskArr[count,0] and  time < secondaryTaskArr[count+1,0]:
                finalArrayEpoc[idx, -1] = 1 if secondaryTaskArr[count, 1] is True else 0
            elif time > secondaryTaskArr[count+1, 0]:
                finalArrayEpoc[idx, -1] = 1 if secondaryTaskArr[count + 1, 1] is True else 0
                count += 1


    #### SHIMMER PRE PROCESSING ####
    #Add Secondary task timestamps to EEG data
    idx = 0
    for i in range(secondaryTaskArr.shape[0]):
        timestamp = secondaryTaskArr[i,0]
        while timestamp > shimmerArr[idx,1]:
            idx += 1
            if idx == finalArrayShimmer.shape[0]:
                break

        if idx == finalArrayShimmer.shape[0]:
            break
        if idx < finalArrayShimmer.shape[0]:
            finalArrayShimmer[idx, -3] = timestamp

    #Add 5 Seconds Windows timestamps to Shimmer data
    idx = 0
    for i in range(windowStampsArr.shape[0]):
        timestamp = windowStampsArr[i,0]
        while timestamp > shimmerArr[idx,1]:
            idx += 1
            if idx == finalArrayShimmer.shape[0]:
                break

        if idx == finalArrayShimmer.shape[0]:
            break
        if idx < finalArrayShimmer.shape[0]:
            finalArrayShimmer[idx, -2] = timestamp

    # Add Label
    count = 0
    for idx in range(finalArrayShimmer.shape[0]):
        time = finalArrayShimmer[idx, 1]

        if count == secondaryTaskArr.shape[0] -1:
            finalArrayShimmer[idx, -1] = 1 if secondaryTaskArr[count, 1] is True else 0
        else:
            if time > secondaryTaskArr[count,0] and  time < secondaryTaskArr[count+1,0]:
                finalArrayShimmer[idx, -1] = 1 if secondaryTaskArr[count, 1] is True else 0
            elif time > secondaryTaskArr[count+1, 0]:
                finalArrayShimmer[idx, -1] = 1 if secondaryTaskArr[count + 1, 1] is True else 0
                count += 1

    # Write Final File
    #EEG File
    with open('./Data/D{:d}/preprocessed/S1_T{:d}_fusion_epoc.txt'.format(dataset, trial),'w') as fout:
        fout.write("AF3 F7 F3 FC5 T7 P7 O1 O2 P8 T8 FC6 F4 F8 AF4 COMPUTER_TIME SECONDARY_TASK 5_SECOND_WINDOW LABEL\n")
        for i in range(finalArrayEpoc.shape[0]):
            formattedData = " ".join(["{:.8f}".format(d) for d in finalArrayEpoc[i]])
            fout.write(formattedData+'\n')
    
    #Shimmer File
    with open('./Data/D{:d}/preprocessed/S1_T{:d}_fusion_shimmer.txt'.format(dataset, trial),'w') as fout:
        fout.write("PPG COMPUTER_TIME SECONDARY_TASK 5_SECOND_WINDOW LABEL\n")
        for i in range(finalArrayShimmer.shape[0]):
            formattedData = " ".join(["{:.8f}".format(d) for d in finalArrayShimmer[i]])
            fout.write(formattedData+'\n')
            
########            
# main##
########

d1 = product([1],[3,4,5,6,7])
d2 = product([2],[2,4,5,6,7])
d3 = product([3],[3,4,5,6,7])
d4 = product([4],[1,2,3,4,5])

completeData = [d1,d2, d3,d4]

for idx, iterT in enumerate(completeData):
    print("Preprocessing dataset {:d}".format(idx + 1))
    for dataset, trial in iterT:
        preprocessData(trial=trial, dataset=dataset)

Preprocessing dataset 1
Preprocess Data from trial 3
Preprocess Data from trial 4
Preprocess Data from trial 5
Preprocess Data from trial 6
Preprocess Data from trial 7
Preprocessing dataset 2
Preprocess Data from trial 2
Preprocess Data from trial 4
Preprocess Data from trial 5
Preprocess Data from trial 6
Preprocess Data from trial 7
Preprocessing dataset 3
Preprocess Data from trial 3
Preprocess Data from trial 4
Preprocess Data from trial 5
Preprocess Data from trial 6
Preprocess Data from trial 7
Preprocessing dataset 4
Preprocess Data from trial 1
Preprocess Data from trial 2
Preprocess Data from trial 3
Preprocess Data from trial 4
Preprocess Data from trial 5


#Create Pickle files

In [7]:
import pandas as pd
import numpy as np
import time
import pickle
from scipy import signal
import cv2
import os

class DataContainer:

    def __init__(self):
        self.dataWindowArray = []
        self.length = 0

        # Normalizing values of whole trial
        self.epocMean = -1
        self.epocStd = -1
        self.shimmerMean = -1
        self.shimmerStd = -1

    def createWindow(self,beginTime, endTime, shimmerSize, epocSize, shimmerScaling=None,
                                                                     epocScaling=None):
        self.dataWindowArray.append(DataWindow(beginTime, endTime, shimmerSize, epocSize,shimmerScaling, epocScaling))
        self.length += 1

    def fillData(self, shimmerData=None, epocData=None):
        #Fill Shimmer Data
        windowIdx = 0
        shimmerIdx = 0
        shimmerData = shimmerData[['PPG', 'COMPUTER_TIME', 'LABEL']].values
        for i in range(shimmerData.shape[0]):
            while shimmerData[i,-2] > self.dataWindowArray[windowIdx].endTime:
                windowIdx += 1
                shimmerIdx = 0

                if windowIdx >= self.length:
                    break

            if windowIdx >= self.length:
                break

            self.dataWindowArray[windowIdx].shimmerArray[shimmerIdx] = shimmerData[i,:]
            self.dataWindowArray[windowIdx].actualShimmerSize += 1
            shimmerIdx += 1

        #Fill Epoc Data
        windowIdx = 0
        epocIdx = 0
        epocData = epocData[['AF3', 'F7', 'F3', 'FC5', 'T7',
                             'P7', 'O1', 'O2', 'P8', 'T8',
                             'FC6', 'F4', 'F8', 'AF4', 'COMPUTER_TIME',
                             'LABEL']].values
       
                    
        for i in range(epocData.shape[0]):
          
            '''
            Get the correct initial window (This needs to be done because the secondary 
            task labels started before that the sensor streaming of data)
            '''
            while epocData[i,-2] > self.dataWindowArray[windowIdx].endTime:
                windowIdx += 1
                epocIdx = 0
            
                if windowIdx >= self.length:
                    break
            
            
            if windowIdx >= self.length:
                break
            
            '''
            Discard data that was taken before the first window of time
            '''
            if epocData[i,-2] > self.dataWindowArray[windowIdx].beginTime:
                self.dataWindowArray[windowIdx].epocArray[epocIdx] = epocData[i,:]
                self.dataWindowArray[windowIdx].actualEpocSize += 1
                epocIdx += 1

    def createMetrics(self):
            
        for i in range(self.length):
            self.dataWindowArray[i].calculateLabel()
            self.dataWindowArray[i].createSpectogramVolume()

    def normalizeData(self):
        for i in range(self.length):
            self.dataWindowArray[i].normalizeData()

    def dumpWindowsToFiles(self, destinationPath, trial):
        for i in range(self.length):
            self.dataWindowArray[i].createPickleFile(i, destinationPath, trial)



class DataWindow:

    def __init__(self, beginTime, endTime, shimmerSize, epocSize,shimmerScaling=None, epocScaling=None):

        self.beginTime = float(beginTime)
        self.endTime = float(endTime)

        self.shimmerArray = np.zeros((shimmerSize, 1+2))
        self.epocArray = np.zeros((epocSize, 14+2))
        self.actualShimmerSize = 0
        self.actualEpocSize = 0

        self.spectogramVolume = np.zeros((14,51,7))
        self.globalLabel = None

        self.shimmerMean = shimmerScaling[0]
        self.shimmerStd = shimmerScaling[1]
        self.epocMean = epocScaling[0]
        self.epocStd = epocScaling[1]


    def createSpectogramVolume(self):
        if self.actualEpocSize > 400:
            #Add EEG Spectogram
            for j in range(14):
                x = self.epocArray[:self.actualEpocSize, j]
                f, t, Sxx = signal.spectrogram(x, 1, nperseg=100, mode='magnitude')
                # print(Sxx.max())
                Sxx = cv2.resize(Sxx, dsize=(7, 51))
                self.spectogramVolume[j, :, :] = Sxx

            # #Add PPG Spectogram
            # if self.actualShimmerSize < 600:
            #     print(self.actualShimmerSize)
            #
            # x = self.shimmerArray[:self.actualShimmerSize, 0]
            # f, t, Sxx = signal.spectrogram(x, 1, nperseg=100, mode='magnitude')
            # Sxx = cv2.resize(Sxx, dsize=(7, 51))
            # self.spectogramVolume[14, :, :] = Sxx


    def normalizeData(self):
        self.epocArray[:,:14] = (self.epocArray[:,:14] - self.epocMean)/self.epocStd
        self.shimmerArray[:,0] = (self.shimmerArray[:,0] - self.shimmerMean)/self.shimmerStd

    def calculateLabel(self):
        totalLength = self.actualEpocSize + 1e-6
        label = sum(self.epocArray[:self.actualEpocSize, -1]) / totalLength
        self.globalLabel = round(label)

    def createPickleFile(self, windowIdx, destinationPath, trial):
        if self.actualEpocSize > 400 and windowIdx>4:
            finalDict = {'data': self.spectogramVolume, 'label': self.globalLabel}
            with open(destinationPath + 'S1_T{:d}_{:03d}.pickle'.format(trial, windowIdx), 'wb') as handle:
                pickle.dump(finalDict, handle, protocol=pickle.HIGHEST_PROTOCOL)


TRIAL = 7
DATASET = 1
import os

d1 = (1,[3,4,5,6,7])
d2 = (2,[2,4,5,6,7])
d3 = (3,[3,4,5,6,7])
d4 = (4,[1,2,3,4,5])
completeData = [d1,d2,d3,d4]

if __name__ == '__main__':
  
    os.chdir('/content/RunTimeDir')
    path = os.getcwd()  
    print ("The current working directory is %s" % path, end='\n\n') 
    
    for DATASET, TRIALS in completeData:
        print("Final preprocessing of D{:d}".format(DATASET))
        
        destinationPath = "./Data/D{:d}/final/".format(DATASET)
        dataPath = "./Data/D{:d}/preprocessed/".format(DATASET)

        try:
            os.mkdir(destinationPath)
        except OSError:
            print("Creation of the directory %s failed" % destinationPath)
        else:
            print("Successfully created the directory %s " % destinationPath)


        for TRIAL in TRIALS:
            shimmerFile = pd.read_csv(dataPath + 'S1_T{:d}_fusion_shimmer.txt'.format(TRIAL), sep=' ')
            epocFile = pd.read_csv(dataPath + 'S1_T{:d}_fusion_epoc.txt'.format(TRIAL), sep=' ')
            fiveSecondWindowStamps = pd.read_csv(dataPath + "S1_T{:d}_TimestampEvery5Seconds.txt".format(TRIAL)
                                                 , sep=' ').values

            #Get Normalizing Values
            shimmerData = shimmerFile['PPG'].values
            epocData = epocFile[['AF3', 'F7', 'F3', 'FC5', 'T7',
                                 'P7', 'O1', 'O2', 'P8', 'T8',
                                 'FC6', 'F4', 'F8', 'AF4']].values

            epocMean, epocStd = epocData.mean(), epocData.std()
            shimmerMean, shimmerStd = shimmerData.mean(), shimmerData.std()

            #Create and fill Data Container
            container = DataContainer()

            for i in range(fiveSecondWindowStamps.shape[0] - 1):
                container.createWindow(fiveSecondWindowStamps[i],
                                       fiveSecondWindowStamps[i+1],
                                       1000, 1000,
                                       shimmerScaling=(shimmerMean, shimmerStd),
                                       epocScaling=(epocMean,epocStd))

            container.fillData(shimmerData=shimmerFile, epocData = epocFile)
            container.normalizeData()
            container.createMetrics()
            container.dumpWindowsToFiles(destinationPath, TRIAL)

            print("Finish Creating Pickle Files from trial {:d}".format(TRIAL))


The current working directory is /content/RunTimeDir

Final preprocessing of D1
Creation of the directory ./Data/D1/final/ failed
Finish Creating Pickle Files from trial 3
Finish Creating Pickle Files from trial 4
Finish Creating Pickle Files from trial 5
Finish Creating Pickle Files from trial 6
Finish Creating Pickle Files from trial 7
Final preprocessing of D2
Creation of the directory ./Data/D2/final/ failed
Finish Creating Pickle Files from trial 2
Finish Creating Pickle Files from trial 4
Finish Creating Pickle Files from trial 5
Finish Creating Pickle Files from trial 6
Finish Creating Pickle Files from trial 7
Final preprocessing of D3
Creation of the directory ./Data/D3/final/ failed
Finish Creating Pickle Files from trial 3
Finish Creating Pickle Files from trial 4
Finish Creating Pickle Files from trial 5
Finish Creating Pickle Files from trial 6
Finish Creating Pickle Files from trial 7
Final preprocessing of D4
Creation of the directory ./Data/D4/final/ failed
Finish Creat

#Test GPU

In [0]:
# import torch
# import pycuda.driver as cuda


# print("Do we have GPUs available?")
# print(torch.cuda.is_available())

# ## Get Id of default device
# cuda.init()
# print("Default device: {:d}".format(torch.cuda.current_device()) )
# print("GPU name: {:s}".format(cuda.Device(0).name()))

# print("GPU name with Pytorch: {:s}".format(torch.cuda.get_device_name(0) ))


#Classification


##Model

In [0]:
import torch.nn as nn

class ConvNetwork(nn.Module):

    def __init__(self, out_1=2, out_2=1, out_3=12, activation="relu"):

        super(ConvNetwork,self).__init__()

        #SET activation
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'selu':
            self.activation = nn.SELU()
        elif activation == 'elu':
            self.activation = nn.ELU()

        #First Convolutional Layer
        self.cnn1 = nn.Conv2d(in_channels=14,out_channels=out_1,kernel_size=1,padding=0)
        #self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.bn1 = nn.BatchNorm2d(out_1)
        self.relu1 = self.activation

        #Second Convolutional Layer
        self.cnn2 = nn.Conv2d(in_channels=out_1, out_channels=out_2, kernel_size=3, padding=2)
        self.bn2 = nn.BatchNorm2d(out_2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu2 = self.activation

        # Third Convolutional Layer
        self.cnn3 = nn.Conv2d(in_channels=out_2, out_channels=out_3, kernel_size=1, padding=0)
        self.bn3 = nn.BatchNorm2d(out_3)
        self.relu3 = self.activation

        #Fully connected Layer
        self.fc1 = nn.Linear(out_3*26*4, 784)
        self.fc_bn = nn.BatchNorm1d(784)
        self.fc2 = nn.Linear(784, 2)
        self.relu4 = self.activation

    def forward(self,x):

        out = self.cnn1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        #out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.bn2(out)
        out = self.maxpool2(out)
        out = self.relu2(out)

        out = self.cnn3(out)
        out = self.bn3(out)
        out = self.relu3(out)

        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.fc_bn(out)
        out = self.relu4(out)

        out = self.fc2(out)

        return out



##EEG dataset Class


In [0]:
from torch.utils.data import Dataset, DataLoader
import torch
from os.path import isfile, join
from os import listdir
import pickle


class EegDataset(Dataset):

    def __init__(self,datasetPaths=["./Dataset/"],transform =None):

        self.datasetPaths = datasetPaths
        self.files = []
        self.len = 0
        
        for datasetPath in self.datasetPaths:
            tempArr = [join(datasetPath, f) for f in listdir(datasetPath) if isfile(join(datasetPath, f))]
            self.files += tempArr
            self.len += len(tempArr)

        self.x = torch.zeros((self.len, 14, 51, 7))
        self.y = torch.zeros(len(self.files),dtype=torch.long)

        self.positiveLength = 0
        self.negativeLength = 0
        self.positiveIdx = []
        self.negativeIdx = []

        for i in range(len(self.files)):

            with open(self.files[i],'rb') as f1:
                dataDict = pickle.load(f1)
                self.x[i, :, :, :] = torch.torch.from_numpy(dataDict['data'])
                self.y[i] = int(dataDict['label'])

                if self.y[i] == 1:
                    self.positiveLength += 1
                    self.positiveIdx.append(i)
                elif self.y[i] == 0:
                    self.negativeLength += 1
                    self.negativeIdx.append(i)


        # Normalizin Values
        self.xMean = self.x.mean()
        self.xStd  = self.x.std()
        self.arbitraryScaling = self.x.max()*3/8
        self.xMax = self.x.max()
        self.xMin = self.x.min()

        # Scaling by (max - min)
        self.x = self.x / (self.xMax - self.xMin + 1e-7)

        #Scaling by mean and std
        #self.x = (self.x - self.xMean) / (self.xStd + 1e-7)

        #Scaling with arbitrary scaling factor
        # self.x = self.x / (self.arbitraryScaling)


        self.transform = transform


    def __getitem__(self, index):

        sample = self.x[index], self.y[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.len


##Classifier

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.optim as optim
from os import listdir
from os.path import isfile, join
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time


def save_checkpoint(optimizer, model, epoch, trLossH, valLossH, valAccH, filename):
    checkpoint_dict = {'optimizer': optimizer.state_dict(),
                       'model': model.state_dict(),
                       'epoch': epoch,
                       'trLossH': trLossH,
                       'valLossH': valLossH,
                       'valAccH': valAccH}

    torch.save(checkpoint_dict, filename)


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename, map_location=DEVICE)
    epoch = checkpoint_dict['epoch']
    trLossH = checkpoint_dict['trLossH']
    valLossH = checkpoint_dict['valLossH']
    valAccH = checkpoint_dict['valAccH']

    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])

    return epoch, trLossH, valLossH, valAccH

def trainModel(model, optimizer, lossFunction, trainLoader, validLoader, fold, modelNumb, verbose=False ):
    #TrainLogger
    valAccuracyHistory = []
    valLossHistory = []
    trLossHistory = []

    #Train Fold
    #Reset Max accuracy
    maxValidationAccuracy = 0.7

    for epoch in range(totalEpochs):

        #Train Model with training set
        model.train()
        for x, y in trainLoader:
            x, y = x.to(DEVICE), y.to(DEVICE)  
            optimizer.zero_grad()
            yHat = model(x)
            loss = lossFunction(yHat, y)
            loss.backward()
            optimizer.step()
            trLossHistory.append(loss.item())


        #Calculate Accuracy with validation set
        correct = 0
        total = 0

        model.eval()
        with torch.no_grad():
            for x, y in validLoader:
                x, y = x.to(DEVICE), y.to(DEVICE)  
                z = model(x)
                yHat = z.argmax(1)
                correct += (yHat == y).sum().item()
                total += y.shape[0]

        accuracy = correct / total
        valAccuracyHistory.append(accuracy)

        #Print Accuracy
        if epoch%20 == 0 and epoch != 0 and verbose:
            #print("Loss in Training in epoch {:d} set:       {:.5f}".format(epoch, loss))
            print("Max validation accuracy in epoch {:d} set: {:.5f}".format(epoch, max(valAccuracyHistory)))

        # Save Check Point - Save only the best per fold
        if accuracy > maxValidationAccuracy:
            maxValidationAccuracy = accuracy
            
            checkpoint_filename = MODELS_DIRECTORY + \
                                  'Model{:d}/classifier-f{:02d}.pkl'.format(modelNumb, fold)
            
            save_checkpoint(optimizer, model, epoch, trLossHistory, valLossHistory, valAccuracyHistory,
                            checkpoint_filename)

            checkpoint_txt = MODELS_DIRECTORY + \
                            'Model{:d}_History/classifier-f{:02d}-e{:03d}-acc{:0.5f}.txt'.format(
                             modelNumb, fold, epoch, maxValidationAccuracy)
                
            with open(checkpoint_txt,'w') as f:
                pass

            maxValidationAccuracy = accuracy
            
    return valAccuracyHistory

def fiveKFoldTraining(trainData, folds, cnn, totalEpochs, modelNumb):

    valAccuracyPlots = []
    loss = 0
    for i in range(5):
        print("Start Training Fold {:d}".format(i))
        #Create Data loaders
        trainLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['train'])
        validLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['val'])

        #Create the model, optimizer and loss function
        torch.manual_seed(RANDOM_SEED)
        model = ConvNetwork( out_1=cnn['CONV_LAYER1'],
                             out_2=cnn['CONV_LAYER2'],
                             out_3=cnn['CONV_LAYER3'],
                             activation=cnn['ACTIVATION'])
        
        model.to(DEVICE)
        lossFunction = nn.CrossEntropyLoss()
        optimizer = optim.SGD(params=model.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)

        #Train
        valAccuracyHistory = trainModel(model, 
                                        optimizer, 
                                        lossFunction, 
                                        trainLoader, 
                                        validLoader, 
                                        i, modelNumb)
        
        maxValAccPerFold =  max(valAccuracyHistory)
        valAccuracyPlots.append(valAccuracyHistory)
        print("Max Validation Accuracy fold {:02d}:        {:0.5f}\n".format(i, maxValAccPerFold))


    #Save data Accuracies
    statsFile = MODELS_DIRECTORY + 'Model{:d}/foldAccuracies.pkl'.format(modelNumb)
    with open(statsFile, 'wb') as f:
        pickle.dump(valAccuracyPlots, f)

    #Print training summary
    for i in range(5):
        print("Max accuracy in fold {:d}: {:.4f}".format(i,max(valAccuracyPlots[i])))

    averageMaxAcc = sum([max(valAccuracyPlots[i]) for i in range(5)])/5
    print("Average Max Accuracy: {:.5f}".format(averageMaxAcc))


def createModelDirectory(modelNumber, checkpointDirectory=True):
    if checkpointDirectory:
        path = MODELS_DIRECTORY + "Model{:02d}".format(modelNumber)
    else:
        path = MODELS_DIRECTORY + "Model{:02d}_History".format(modelNumber)
        
    try:
        os.makedirs(path)
    except OSError:
        print("Creation of the directory %s failed" % path)
    else:
        print("Successfully created the directory %s " % path)

        
def get5Folds(trainDataSize):

    #Shuffle training data
    split = int(0.2 * trainDataSize)
    np.random.seed(seed=RANDOM_SEED)
    idxArr = np.random.permutation(trainDataSize)

    #Create 5 folds for cross Validation
    folds = []
    total = trainDataSize

    for i in range(5):
        valRange = range(split*i, split*(i+1) if split*(i+1) < total else total)
        valIdx = idxArr[valRange]
        trainIdx = np.delete(idxArr, valRange)
        trainSampler = SubsetRandomSampler(trainIdx)
        valSampler = SubsetRandomSampler(valIdx)
        folds.append({'train': trainSampler, 'val': valSampler})

    return folds

#Cnn Architectures -- Best Model cnn8!!
cnn1 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'selu'}
cnn2 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'elu'}
cnn3 = {"CONV_LAYER1": 32, "CONV_LAYER2": 64, "CONV_LAYER3": 12, "ACTIVATION": 'relu'}
cnn4 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'selu'}
cnn5 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'elu'}
cnn6 = {"CONV_LAYER1": 16, "CONV_LAYER2": 32, "CONV_LAYER3": 6, "ACTIVATION": 'relu'}
cnn7 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'selu'}
cnn8 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'elu'}
cnn9 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'relu'}


cnn10 = {"CONV_LAYER1": 128, "CONV_LAYER2": 256, "CONV_LAYER3": 48, "ACTIVATION": 'elu'}

#Global variables
DEVICE = 'cuda'
MODELS_DIRECTORY = './Models/'

#RandomSeed
#Model 2 and Model 3
#RANDOM_SEED = 742
#Model 4
RANDOM_SEED = 480


if __name__ == '__main__':

    #Create Training data object
    paths1 = ["./Data/D1/final", "./Data/D2/final", "./Data/D3/final"]
    paths2 = ["./Data/D2/final", "./Data/D3/final", "./Data/D4/final"]
    paths3 = ["./Data/D1/final", "./Data/D2/final", "./Data/D4/final"]
    paths4 = ["./Data/D1/final", "./Data/D3/final", "./Data/D4/final"]
    paths = [paths1,paths2,paths3,paths4]
    
    #trainData = EegDataset(datasetPaths=paths1)

    #Train Models
    totalEpochs = 200
    print("Start training")
    #cnnArch = [cnn1,cnn2,cnn3,cnn4,cnn5,cnn6,cnn7,cnn8,cnn9]
    cnnArch = [cnn8]
    #cnnArch = [cnn10]
    modelNumber = 70

#   #Training Loop
#     for path in paths:  
#         #Load Data
#         trainData = EegDataset(datasetPaths=path)
#         folds = get5Folds(len(trainData))
#         #Print Information
#         print("Start Training Model {:d}".format(modelNumber))
#         print("Number of negative examples ",trainData.negativeLength)
#         print("Number of positive examples ", trainData.positiveLength)
#         print("Dataset size ", len(trainData))
#         #Create Directories
#         createModelDirectory(modelNumber, checkpointDirectory=True)
#         createModelDirectory(modelNumber, checkpointDirectory=False)
#         #Train Model
#         fiveKFoldTraining(trainData, folds, cnn8, totalEpochs, modelNumber)
#         modelNumber += 1



  

Start training
Start Training Model 70
Number of negative examples  559
Number of positive examples  526
Dataset size  1085
Successfully created the directory ./Models/Model70 
Successfully created the directory ./Models/Model70_History 
Start Training Fold 0
Max Validation Accuracy fold 00:        0.80184

Start Training Fold 1
Max Validation Accuracy fold 01:        0.78802

Start Training Fold 2
Max Validation Accuracy fold 02:        0.76037

Start Training Fold 3
Max Validation Accuracy fold 03:        0.75576

Start Training Fold 4
Max Validation Accuracy fold 04:        0.75576

Max accuracy in fold 0: 0.8018
Max accuracy in fold 1: 0.7880
Max accuracy in fold 2: 0.7604
Max accuracy in fold 3: 0.7558
Max accuracy in fold 4: 0.7558
Average Max Accuracy: 0.77235
Start Training Model 71
Number of negative examples  616
Number of positive examples  552
Dataset size  1168
Successfully created the directory ./Models/Model71 
Successfully created the directory ./Models/Model71_History 

#Copy Models to Drive


In [0]:
import os

os.chdir('/content')

dstPath = "./gdrive/My Drive/ToshibaJuanAntonio/"+\
       "Trabajos Purdue/Surgical Intuitive Grant/"+\
       "MentalWorkloadPrediction/Models/"

dstPath = dstPath+"Trial4/"
srcPath = "/content/RunTimeDir/Models/"


try:
    os.makedirs(dstPath)
except OSError:
    print("Creation of the directory %s failed" % dstPath)
else:
    print("Successfully created the directory %s " % dstPath)

#Copy models folder 
from distutils.dir_util import copy_tree
copy_tree(srcPath, dstPath)


os.chdir('/content/RunTimeDir')

Successfully created the directory ./gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Models/Trial4/ 


#Load Models to Colab


In [0]:
import os
import shutil

def copyWholeDirectory(srcPath,dstPath):
    directoriesToCopy = os.listdir(srcPath)

    for p in directoriesToCopy:
        if os.path.isdir(srcPath+p):
            copyWholeDirectory(srcPath+p+'/',dstPath+p+'/')
        else:
            try:
                os.makedirs(dstPath)
            except OSError:
                pass
                #print("Creation of the directory %s failed" % dstPath)
            else:
                pass
                #print("Successfully created the directory %s " % dstPath)

            shutil.copy(srcPath+p, dstPath)

            
os.chdir('/content')

#Remove Previous ModelsDirectory
try:
    shutil.rmtree('./RunTimeDir/Models')
except:
    print("Directory not found")
else:
    print("Deleting previous Directory")
finally:
    print()

srcPath = "./gdrive/My Drive/ToshibaJuanAntonio/"+\
       "Trabajos Purdue/Surgical Intuitive Grant/"+\
       "MentalWorkloadPrediction/Models/"

srcPath = srcPath+"Trial2/"

dstPath = "./RunTimeDir/Models/"


try:
    os.makedirs(dstPath)
except OSError:
    print("Creation of the directory %s failed" % dstPath)
else:
    print("Successfully created the directory %s " % dstPath)

#Copy models folder 
print("Copy models from drive")
copyWholeDirectory(srcPath, dstPath)


os.chdir('/content/RunTimeDir')



Deleting previous Directory

Successfully created the directory ./RunTimeDir/Models/ 
Copy models from drive


In [0]:
# os.chdir('/content')
# src='./gdrive/My Drive/ToshibaJuanAntonio/Trabajos Purdue/Surgical Intuitive Grant/MentalWorkloadPrediction/Models/Trial1/Model40/classifier-f03.pkl'

# dst='./RunTimeDir/Models/Model40/'
# os.mkdir(dst)
# shutil.copy(src, dst)


#Testing Module



In [0]:
import numpy as np


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename, map_location=DEVICE)
    epoch = checkpoint_dict['epoch']
    trLossH = checkpoint_dict['trLossH']
    valLossH = checkpoint_dict['valLossH']
    valAccH = checkpoint_dict['valAccH']

    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])

    return epoch, trLossH, valLossH, valAccH


def calculateAcc(model, loader):
        correct = 0
        total = 0

        model.eval()
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(DEVICE), y.to(DEVICE)  
                z = model(x)
                yHat = z.argmax(1)
                correct += (yHat == y).sum().item()
                total += y.shape[0]

        accuracy = correct / total
        return accuracy

#Global Variables

cnn8 = {"CONV_LAYER1": 64, "CONV_LAYER2": 128, "CONV_LAYER3": 24, "ACTIVATION": 'elu'}
DEVICE ='cuda'
RANDOM_SEED = 480
 
if __name__ == "__main__":
    
    print("Test Model on test set")
    
    modelNumber = 50
    pathToClassifiers = "./Models/Model{:02d}/".format(modelNumber)
    pathToTestData  = "./Data/D4/final"
    pathToTrainData = ["./Data/D1/final", "./Data/D2/final", "./Data/D3/final"]
    accDict={'train':[], 'val':[], 'test':[]}
    
    #Load test Dataset
    testData = EegDataset(datasetPaths=[pathToTestData])
    testLoader = DataLoader(trainData, batch_size=100, shuffle=True )
    print("Test Negative: ", testData.negativeLength)
    print("Test Positive: ", testData.positiveLength)
    print("Test set Accuracy:")
    
    #Load train dataset
    trainData = EegDataset(datasetPaths=path)
    folds = get5Folds(len(trainData))
    
    for fold in range(5):
        #Get train and valid loaders
        trainLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['train'])
        validLoader = DataLoader(trainData, batch_size=16, sampler=folds[i]['val'])
        
        #Load Model
        torch.manual_seed(RANDOM_SEED)
        model = ConvNetwork( out_1=cnn8['CONV_LAYER1'],
                             out_2=cnn8['CONV_LAYER2'],
                             out_3=cnn8['CONV_LAYER3'],
                             activation=cnn8['ACTIVATION'])

        optimizer = optim.SGD(params=model.parameters(), 
                              lr=0.01, 
                              weight_decay=1e-6, 
                              momentum=0.9, 
                              nesterov=True)

        load_checkpoint(optimizer, model, pathToClassifiers+'classifier-f{:02d}.pkl'.format(fold))
        model.to(DEVICE)

        #Calculate Accuracies
        accuracyVal = calculateAcc(model,testLoader)
        accDict['test'] += [accuracyVal]
        print("\tfold{:02d}: {:0.5f}".format(fold, accuracyVal))
        
   
    print("Average Accuracy: {:0.5f}".format(np.mean(accDict['test'])))


Test Model on test set


NameError: ignored